## Renewable Energy requirements prediction project

### Context:
    This will act as the base data for the investigation into the possible solutions for the UK energy requirements

### Content:
     Data from the UK statistics on renewable energy generation.

Acknowledgements
https://www.gov.uk/government/statistics/regional-renewable-statistics7

In [79]:
from pyspark.sql import SparkSession

In [80]:
spark = SparkSession.builder.appName('energy').getOrCreate()

In [81]:
df = spark.read.csv('../datasets/renewable_energy.csv',inferSchema=True,header=True)

In [82]:
df.show()

+---+----+--------------------+-----+--------------+--------+-------+------------+---------------------------------+-------+
|_c0|Year|              Region|Wind2|Wave and tidal|Solar PV|  Hydro|Landfill gas|Other bioenergy (incl Sewage gas3|  Total|
+---+----+--------------------+-----+--------------+--------+-------+------------+---------------------------------+-------+
|  1|2003|             England|349.2|           0.0|     0.0|   25.3|     2,899.0|                          2,716.9|5,990.4|
|  2|2003|       East Midlands|  1.3|           0.0|     0.0|    5.5|       223.1|                            202.7|  432.6|
|  3|2003|     East of England| 19.8|           0.0|     0.0|      -|       756.4|                            748.8|1,525.0|
|  4|2003|          North East| 28.6|           0.0|     0.0|      …|        89.9|                            139.0|  257.5|
|  5|2003|          North West|120.8|           0.0|     0.0|    2.0|       519.4|                            216.3|  858.5|


In [83]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Wind2: string (nullable = true)
 |-- Wave and tidal: double (nullable = true)
 |-- Solar PV: string (nullable = true)
 |-- Hydro: string (nullable = true)
 |-- Landfill gas: string (nullable = true)
 |-- Other bioenergy (incl Sewage gas3: string (nullable = true)
 |-- Total: string (nullable = true)



#### Checking for NAN values

In [84]:
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(df[k].isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [85]:
null_values = null_value_count(df)
print("There are %s null data points in dataset"%len(null_values))

There are 0 null data points in dataset


### Data preprocessing and cleaning:
    replace '…' and '-' to 0
    and removing ',' from numerical values

In [86]:
df = df.replace('…','0')
df = df.replace('-','0')
df.show()

+---+----+--------------------+-----+--------------+--------+-------+------------+---------------------------------+-------+
|_c0|Year|              Region|Wind2|Wave and tidal|Solar PV|  Hydro|Landfill gas|Other bioenergy (incl Sewage gas3|  Total|
+---+----+--------------------+-----+--------------+--------+-------+------------+---------------------------------+-------+
|  1|2003|             England|349.2|           0.0|     0.0|   25.3|     2,899.0|                          2,716.9|5,990.4|
|  2|2003|       East Midlands|  1.3|           0.0|     0.0|    5.5|       223.1|                            202.7|  432.6|
|  3|2003|     East of England| 19.8|           0.0|     0.0|      0|       756.4|                            748.8|1,525.0|
|  4|2003|          North East| 28.6|           0.0|     0.0|      0|        89.9|                            139.0|  257.5|
|  5|2003|          North West|120.8|           0.0|     0.0|    2.0|       519.4|                            216.3|  858.5|


In [87]:
df.columns

['_c0',
 'Year',
 'Region',
 'Wind2',
 'Wave and tidal',
 'Solar PV',
 'Hydro',
 'Landfill gas',
 'Other bioenergy (incl Sewage gas3',
 'Total']

`Replacing ',' to '' from all numerical columns`

In [88]:
from pyspark.sql.functions import regexp_replace,col

In [89]:
data = df.withColumn('Total_',regexp_replace(col('Total'),',','')).withColumn('Landfill_gas',regexp_replace(col('Landfill gas'),',','')).withColumn('Other',regexp_replace(col('Other bioenergy (incl Sewage gas3'),',','')).withColumn('Solar_PV',regexp_replace(col('Solar PV'),',','')).withColumn('Wind_2',regexp_replace(col('Wind2'),',','')).withColumn('Hydro_',regexp_replace(col('Hydro'),',',''))


In [90]:
data.select('Hydro','Hydro_').show()

+-------+------+
|  Hydro|Hydro_|
+-------+------+
|   25.3|  25.3|
|    5.5|   5.5|
|      0|     0|
|      0|     0|
|    2.0|   2.0|
|      0|     0|
|      0|     0|
|   16.4|  16.4|
|    1.4|   1.4|
|      0|     0|
|    6.7|   6.7|
|2,902.0|2902.0|
|  194.7| 194.7|
|    8.8|   8.8|
|   70.2|  70.2|
|   11.1|  11.1|
|      0|     0|
|   16.9|  16.9|
|    8.2|   8.2|
|      0|     0|
+-------+------+
only showing top 20 rows



In [91]:
columns_to_drop = ['Total','Landfill gas','Other bioenergy (incl Sewage gas3','Solar PV','Wind2','Hydro','_c0']
data = data.drop(*columns_to_drop)
data.show()

+----+--------------------+--------------+------+------------+------+--------+------+------+
|Year|              Region|Wave and tidal|Total_|Landfill_gas| Other|Solar_PV|Wind_2|Hydro_|
+----+--------------------+--------------+------+------------+------+--------+------+------+
|2003|             England|           0.0|5990.4|      2899.0|2716.9|     0.0| 349.2|  25.3|
|2003|       East Midlands|           0.0| 432.6|       223.1| 202.7|     0.0|   1.3|   5.5|
|2003|     East of England|           0.0|1525.0|       756.4| 748.8|     0.0|  19.8|     0|
|2003|          North East|           0.0| 257.5|        89.9| 139.0|     0.0|  28.6|     0|
|2003|          North West|           0.0| 858.5|       519.4| 216.3|     0.0| 120.8|   2.0|
|2003|              London|           0.0| 438.8|           0| 438.8|     0.0|     0|     0|
|2003|          South East|           0.0| 792.8|       602.3| 187.0|     0.0|   3.5|     0|
|2003|          South West|           0.0| 452.6|       249.7|  90.7| 

In [92]:
data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Wave and tidal: double (nullable = true)
 |-- Total_: string (nullable = true)
 |-- Landfill_gas: string (nullable = true)
 |-- Other: string (nullable = true)
 |-- Solar_PV: string (nullable = true)
 |-- Wind_2: string (nullable = true)
 |-- Hydro_: string (nullable = true)



### Converting all Numerical columns to double type (it's String now)

In [93]:
from pyspark.sql.types import DoubleType

In [94]:
final_data = data.withColumn('Total_',data['Total_'].cast(DoubleType())).withColumn('Landfill_gas',data['Landfill_gas'].cast(DoubleType())).withColumn('Other',data['Other'].cast(DoubleType())).withColumn('Solar_PV',data['Solar_PV'].cast(DoubleType())).withColumn('Wind_2',data['Wind_2'].cast(DoubleType())).withColumn('Hydro_',data['Hydro_'].cast(DoubleType()))

final_data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Wave and tidal: double (nullable = true)
 |-- Total_: double (nullable = true)
 |-- Landfill_gas: double (nullable = true)
 |-- Other: double (nullable = true)
 |-- Solar_PV: double (nullable = true)
 |-- Wind_2: double (nullable = true)
 |-- Hydro_: double (nullable = true)



#### Indexing my Region column to numerical column

In [95]:
from pyspark.ml.feature import StringIndexer,VectorAssembler

In [96]:
indexer = StringIndexer(inputCol='Region',outputCol='Region_index')

In [97]:
final_data = indexer.fit(final_data).transform(final_data)

In [98]:
final_data.select('Region','Region_index').show()

+--------------------+------------+
|              Region|Region_index|
+--------------------+------------+
|             England|        12.0|
|       East Midlands|        13.0|
|     East of England|         1.0|
|          North East|         9.0|
|          North West|         7.0|
|              London|         0.0|
|          South East|         8.0|
|          South West|         2.0|
|       West Midlands|         3.0|
|Yorkshire and the...|         5.0|
|    Northern Ireland|        10.0|
|            Scotland|         6.0|
|               Wales|         4.0|
|        Other Sites4|        11.0|
|             England|        12.0|
|       East Midlands|        13.0|
|     East of England|         1.0|
|          North East|         9.0|
|          North West|         7.0|
|              London|         0.0|
+--------------------+------------+
only showing top 20 rows



In [99]:
final_data = final_data.drop('Region')
final_data.columns

['Year',
 'Wave and tidal',
 'Total_',
 'Landfill_gas',
 'Other',
 'Solar_PV',
 'Wind_2',
 'Hydro_',
 'Region_index']

#### Assemble the features

In [100]:
assembler = VectorAssembler(inputCols=['Year','Wave and tidal','Landfill_gas',
 'Other',
 'Solar_PV',
 'Wind_2',
 'Hydro_',
 'Region_index'],outputCol='features')

In [101]:
final_data = assembler.transform(final_data)

#### Selecting only features and label column

In [102]:
final_data = final_data.select('Total_','features')
final_data.show()

+------+--------------------+
|Total_|            features|
+------+--------------------+
|5990.4|[2003.0,0.0,2899....|
| 432.6|[2003.0,0.0,223.1...|
|1525.0|[2003.0,0.0,756.4...|
| 257.5|[2003.0,0.0,89.9,...|
| 858.5|[2003.0,0.0,519.4...|
| 438.8|(8,[0,3],[2003.0,...|
| 792.8|[2003.0,0.0,602.3...|
| 452.6|[2003.0,0.0,249.7...|
| 581.3|[2003.0,0.0,224.6...|
| 651.3|[2003.0,0.0,233.6...|
| 104.3|[2003.0,0.0,0.0,1...|
|3724.4|[2003.0,0.0,228.0...|
| 768.9|[2003.0,0.0,149.3...|
|  11.7|(8,[0,4,6,7],[200...|
|7119.6|[2004.0,0.0,3501....|
| 377.3|[2004.0,0.0,287.2...|
|1574.0|[2004.0,0.0,835.9...|
| 405.5|[2004.0,0.0,132.3...|
|1068.0|[2004.0,0.0,687.9...|
| 434.8|(8,[0,3],[2004.0,...|
+------+--------------------+
only showing top 20 rows



#### Normalizing the data

In [103]:
from pyspark.ml.feature import StandardScaler

In [104]:
scaler = StandardScaler(inputCol='features',outputCol='scaled_features')

### Scaling Total column

In [105]:
from pyspark.sql.functions import mean,stddev

mean_total, sttdev_total = final_data.select(mean("Total_"), stddev("Total_")).first()
final_data=final_data.withColumn("scaled_total", (col("Total_") - mean_total) / sttdev_total)

In [106]:
final_data.show()

+------+--------------------+--------------------+
|Total_|            features|        scaled_total|
+------+--------------------+--------------------+
|5990.4|[2003.0,0.0,2899....|  1.5486085855875653|
| 432.6|[2003.0,0.0,223.1...| -0.3823508034399062|
|1525.0|[2003.0,0.0,756.4...|-0.00281572902671...|
| 257.5|[2003.0,0.0,89.9,...|-0.44318620396146335|
| 858.5|[2003.0,0.0,519.4...|-0.23437937521528493|
| 438.8|(8,[0,3],[2003.0,...|-0.38019672301024515|
| 792.8|[2003.0,0.0,602.3...|-0.25720567912314507|
| 452.6|[2003.0,0.0,249.7...|-0.37540215689261236|
| 581.3|[2003.0,0.0,224.6...| -0.3306876163607769|
| 651.3|[2003.0,0.0,233.6...| -0.3063673534452486|
| 104.3|[2003.0,0.0,0.0,1...| -0.4964128365137338|
|3724.4|[2003.0,0.0,228.0...|   0.761326931779179|
| 768.9|[2003.0,0.0,149.3...|-0.26550931174716114|
|  11.7|(8,[0,4,6,7],[200...|  -0.528585070027704|
|7119.6|[2004.0,0.0,3501....|  1.9409291696477728|
| 377.3|[2004.0,0.0,287.2...| -0.4015638111431736|
|1574.0|[2004.0,0.0,835.9...|0.

### Now scaling our features

In [107]:
final_data = scaler.fit(final_data).transform(final_data)
final_data.show()

+------+--------------------+--------------------+--------------------+
|Total_|            features|        scaled_total|     scaled_features|
+------+--------------------+--------------------+--------------------+
|5990.4|[2003.0,0.0,2899....|  1.5486085855875653|[533.851570764226...|
| 432.6|[2003.0,0.0,223.1...| -0.3823508034399062|[533.851570764226...|
|1525.0|[2003.0,0.0,756.4...|-0.00281572902671...|[533.851570764226...|
| 257.5|[2003.0,0.0,89.9,...|-0.44318620396146335|[533.851570764226...|
| 858.5|[2003.0,0.0,519.4...|-0.23437937521528493|[533.851570764226...|
| 438.8|(8,[0,3],[2003.0,...|-0.38019672301024515|(8,[0,3],[533.851...|
| 792.8|[2003.0,0.0,602.3...|-0.25720567912314507|[533.851570764226...|
| 452.6|[2003.0,0.0,249.7...|-0.37540215689261236|[533.851570764226...|
| 581.3|[2003.0,0.0,224.6...| -0.3306876163607769|[533.851570764226...|
| 651.3|[2003.0,0.0,233.6...| -0.3063673534452486|[533.851570764226...|
| 104.3|[2003.0,0.0,0.0,1...| -0.4964128365137338|[533.851570764

#### Keep only two decimals after point in scaled_total

In [132]:
from pyspark.sql.functions import round

final_data = final_data.withColumn('scaled_total', round(lr_pred['scaled_total'],2))
final_data = final_data.drop('Total_','features')
final_data.show()

+------------+--------------------+
|scaled_total|     scaled_features|
+------------+--------------------+
|        1.55|[533.851570764226...|
|       -0.38|[533.851570764226...|
|         0.0|[533.851570764226...|
|       -0.44|[533.851570764226...|
|       -0.23|[533.851570764226...|
|       -0.38|(8,[0,3],[533.851...|
|       -0.26|[533.851570764226...|
|       -0.38|[533.851570764226...|
|       -0.33|[533.851570764226...|
|       -0.31|[533.851570764226...|
|        -0.5|[533.851570764226...|
|        0.76|[533.851570764226...|
|       -0.27|[533.851570764226...|
|       -0.53|(8,[0,4,6,7],[533...|
|        1.94|[534.118096760613...|
|        -0.4|[534.118096760613...|
|        0.01|[534.118096760613...|
|       -0.39|[534.118096760613...|
|       -0.16|[534.118096760613...|
|       -0.38|(8,[0,3],[534.118...|
+------------+--------------------+
only showing top 20 rows



#### Train Test Split

In [133]:
train_data, test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-------------------+
|summary|       scaled_total|
+-------+-------------------+
|  count|                125|
|   mean|0.00799999999999995|
| stddev| 1.0582418468628108|
|    min|              -0.53|
|    max|               6.93|
+-------+-------------------+



### Model building

In [134]:
from pyspark.ml.regression import LinearRegression,RandomForestRegressor

In [135]:
lr = LinearRegression(featuresCol='scaled_features',labelCol='scaled_total')
rf = RandomForestRegressor(featuresCol='scaled_features',labelCol='scaled_total')

In [136]:
lr_model = lr.fit(train_data)
rf_model = rf.fit(train_data)

In [137]:
lr_pred = lr_model.transform(test_data)
rf_pred = rf_model.transform(test_data)

In [144]:
rf_pred = rf_pred.withColumn('prediction',round(rf_pred['prediction'],2))
lr_pred = lr_pred.withColumn('prediction',round(lr_pred['prediction'],2))
rf_pred.show()

+------------+--------------------+----------+
|scaled_total|     scaled_features|prediction|
+------------+--------------------+----------+
|       -0.53|[536.250304731714...|      -0.5|
|       -0.52|[534.651148753388...|      -0.5|
|       -0.52|[535.717252738939...|     -0.37|
|       -0.51|[536.516830728101...|     -0.48|
|       -0.48|[534.118096760613...|     -0.49|
|       -0.48|[536.250304731714...|      -0.3|
|       -0.45|[535.184200746163...|     -0.18|
|       -0.44|[533.851570764226...|     -0.27|
|       -0.44|[536.516830728101...|     -0.31|
|       -0.43|[536.516830728101...|     -0.24|
|       -0.42|[535.184200746163...|      -0.2|
|       -0.42|[535.717252738939...|     -0.36|
|       -0.42|[535.983778735326...|     -0.37|
|       -0.41|[534.651148753388...|     -0.38|
|       -0.41|[535.450726742551...|     -0.32|
|       -0.41|[535.450726742551...|      -0.2|
|        -0.4|[534.118096760613...|     -0.16|
|       -0.39|[534.118096760613...|     -0.11|
|       -0.38

In [145]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

In [146]:
eval = BinaryClassificationEvaluator(labelCol='scaled_total',rawPredictionCol='prediction')
acc_eval = MulticlassClassificationEvaluator(labelCol='scaled_total',metricName='accuracy',predictionCol='prediction')

In [147]:
eval.evaluate(lr_pred)

0.8856209150326798

In [148]:
eval.evaluate(rf_pred)

0.9738562091503268

In [151]:
acc_eval.evaluate(rf_pred)

0.017543859649122806